In [1]:
# Load the tfa model 
from pytfa.io.json import load_json_model
model_file = 'reduced_model_ETC_core_20240816-155234_continuous.json'
tmodel = load_json_model(model_file)
#sol = tmodel.optimize()

2025-02-22 09:17:32,584 - thermomodel_Recon3thermoCurated - continuous - INFO - # Model initialized with units kcal/mol and temperature 298.15 K


In [2]:
# Reload and prepare the model
from skimpy.io.yaml import load_yaml_model
from skimpy.utils.tabdict import TabDict

kmodel = load_yaml_model(model_file.replace("_continuous.json", "_kinetic_curated.yml"))


# Compile the ode expressions
NCPU = 12
kmodel.prepare()
kmodel.compile_ode(ncpu=NCPU)

In [3]:
# Scaling parameters
CONCENTRATION_SCALING = 1e3 # 1 mol to 1 mmol
TIME_SCALING = 1 # 1min
DENSITY = 1200 # g/L 
GDW_GWW_RATIO = 1.0 # Fluxes are in gWW

# To test how close to zero the dxdt is
flux_scaling_factor = 1e-6 / (GDW_GWW_RATIO / DENSITY) \
                        * CONCENTRATION_SCALING \
                        / TIME_SCALING

In [4]:
# Load TFA samples 
import pandas as pd
import numpy as np
tfa_sample_file = 'reduced_model_ETC_core_20240816-155234_tfa_sampling.csv'
tfa_samples = pd.read_csv(tfa_sample_file)

# Add additional fluxes and concentration to model insulin turnover
# Turnover of inusnlin is about 5 min 

tfa_samples['Insulin_secretion'] = 1/5 / flux_scaling_factor
tfa_samples['Insulin_degradation'] = 1/5 / flux_scaling_factor
tfa_samples['insulin_e'] = 1e-3 

tfa_samples['Stimulation_on'] = 1 / flux_scaling_factor
tfa_samples['Stimulation_off'] = 1 / flux_scaling_factor
tfa_samples['stim_e'] = 1e-3



additional_fluxes = ['Insulin_secretion', 'Insulin_degradation', 'Stimulation_on', 'Stimulation_off',]
additional_concentrations = ['insulin_e', 'stim_e', ]



In [5]:
# Load only the robust parameters samples
from skimpy.core.parameters import load_parameter_population
parameter_population = load_parameter_population(tfa_sample_file.replace(".csv",'_robust_parameters.hdf5'))

In [6]:
# Build function to compute fluxes from concentrations
from skimpy.analysis.ode.utils import make_flux_fun
from skimpy.utils.namespace import QSSA

flux_function = make_flux_fun(kmodel, QSSA)

In [7]:
from skimpy.core.parameters import ParameterValues
from skimpy.analysis.oracle import load_concentrations

from tqdm import tqdm

# Function for population analysis
def steady_state_perturbation(parameter, range=np.logspace(-1,1,51,base=10) , 
                            additional_parameter_changes={},
                            time = [0,500], # 1000 min to make sure to reach steady state
                            parameter_population=parameter_population, 
                            kmodel=kmodel, 
                            flux_function=flux_function,
                            tfa_samples=tfa_samples, 
                            end_ix=-1,
                            ):
    
    # List of dataframes
    results = []

    # Run the perturbation for each parameterset
    parameter_population_index = list(parameter_population._index.keys())
    for parameter_set_id in tqdm(parameter_population_index[:end_ix]):

        # This will be a list of dataframes
        this_results = []

        thermo_index = int(parameter_set_id.split(',')[0])
        thermo_sample = tfa_samples.loc[thermo_index]

        concentrations = load_concentrations(thermo_sample, tmodel, kmodel, 
                                            concentration_scaling=CONCENTRATION_SCALING,
                                            additional_concentrations=additional_concentrations)

        for k in kmodel.initial_conditions:
                kmodel.initial_conditions[k] = concentrations[k]

        # Load the parameter values 
        kmodel.parameters = parameter_population[parameter_set_id]

        # Integrate additional parameters changes (fold changes)
        for k, v in additional_parameter_changes.items():
            kmodel.parameters[k].value = kmodel.parameters[k].value * v

        # Perturb the main parameter
        p0 = kmodel.parameters[parameter].value
        
        for perturbation_value in range:
            kmodel.parameters[parameter].value = perturbation_value * p0
            # Dynamic solution            
            sol = kmodel.solve_ode(time, solver_type='cvode' , max_steps=1e9, rtol=1e-6)
            # Get steady state concentrations
            steady_state_concentrations = sol.concentrations.iloc[-1]

            # Compute the fluxes at steady state
            # Get parameters value set 
            parameter_values = {p.symbol:p.value for p in kmodel.parameters.values()}
            parameter_values = ParameterValues(parameter_values, kmodel)

            steady_state_fluxes = pd.Series(flux_function(steady_state_concentrations, parameters=parameter_values), 
                                            index=kmodel.reactions.keys())
            
            # Add the results to a dataframe 
            steady_state_output = pd.concat([steady_state_concentrations, steady_state_fluxes])
            # Add metadata 
            steady_state_output['pertubration'] = perturbation_value
            steady_state_output['parameter_set'] = parameter_set_id
            steady_state_output['parameter'] = parameter    
            steady_state_output['aditional_parameters'] = [f'{k}:{v:.1f}' for k,v in additional_parameter_changes.items()]

            this_results.append(steady_state_output)

        # Concatenate the results
        this_results = pd.concat(this_results, axis=1).T

        results.append(this_results)
    

    # Concatenate the results
    results = pd.concat(results)

    return results


In [8]:
# Checkpoint -2  - import robust parameters population 
from skimpy.core.parameters import load_parameter_population
parameter_population_robust= load_parameter_population(tfa_sample_file.replace(".csv",'_robust_parameters.hdf5'))

In [9]:
# Checkpint 1 - Simulate the steady state perturbations for 5 fold changes in fuel availability and atp requirement
# This will take a while

parameters = [ 'lac_L_e', 'glc_D_e', 'hdca_e', 'bhb_e', ]
results = [steady_state_perturbation(p, parameter_population=parameter_population_robust) 
                                     for p in parameters]
results = pd.concat(results)

 # Save the results
name = model_file.replace('.json', '')
results.to_csv(f'output/{name}_steady_state_fuel_competition_10_fold.csv')


100%|██████████| 409/409 [14:32<00:00,  2.13s/it]


In [10]:
# Fed levels of insulin
# Compute perturbations for each fuel concentration
parameters = ['lac_L_e', 'glc_D_e', 'hdca_e', 'bhb_e']
model_fed_insulin = {'vmax_forward_Insulin_secretion': 10 , }

results_insulin = [steady_state_perturbation(p, additional_parameter_changes=model_fed_insulin, 
                                             parameter_population=parameter_population_robust) 
                                             for p in parameters]
results_insulin = pd.concat(results_insulin)

100%|██████████| 409/409 [16:42<00:00,  2.45s/it]


In [11]:
# Save the results
name = model_file.replace('.json', '')
results_insulin.to_csv(f'output/{name}_steady_state_fuel_competition_10_fold_fed_insulin.csv')